# Collect NISMOD2 results for NIC resilience - demand scenarios

- water demand
- energy demand
- transport OD matrix, trip distribution, energy consumption

In [ ]:
import glob
import os
import re

import pandas
import geopandas

from pandas.api.types import CategoricalDtype
from tqdm import tqdm

## Water demand

In [ ]:
water_demand_files = glob.glob("../results/nic_w*/water_demand/decision_0/*.csv")
dfs = []
for fn in water_demand_files:
    demand_scenario = re.search("__(\w+)", fn).group(1)
    year = re.search("2\d+", fn).group(0)
    df = pandas.read_csv(fn, dtype={
        'water_resource_zones': 'category'
    })
    df['timestep'] = int(year)    
    df.timestep = df.timestep.astype('int16')
    df['demand_scenario'] = demand_scenario
    df.demand_scenario = df.demand_scenario.astype(CategoricalDtype(['BL', 'FP']))
    dfs.append(df)

water_demand = pandas.concat(dfs)
del dfs
water_demand.head()

In [ ]:
water_demand.dtypes

In [ ]:
water_demand.to_parquet('nic_water_demand.parquet')

## Energy demand

In [ ]:
energy_demand_files = glob.glob("../results/nic_ed/energy_demand_constrained/decision_0/*2050.csv")
dfs = []
for n, fn in enumerate(tqdm(energy_demand_files)):
    output = re.search("output_(\w+)_timestep", fn).group(1)
    year = re.search("2\d+", fn).group(0)
    sector = re.match("[^_]*", output).group(0)
    service = output.replace(sector + "_", "")
    fuel = re.match("hydrogen|oil|solid_fuel|gas|electricity|biomass", service).group(0)
    service = service.replace(fuel + "_", "")
    service = service.replace("_" + fuel, "")
    service = service.replace("_CHP", "")
    service = service.replace("_fuel_cell", "")
    df = pandas.read_csv(fn, dtype={
        'hourly': 'int16',
        'lad_uk_2016': 'category'
    }).rename(columns={
        output: 'energy_demand'
    })
    df['timestep'] = int(year)
    df.timestep = df.timestep.astype('int16')
    df['fuel'] = fuel
    df.fuel = df.fuel.astype(CategoricalDtype(['hydrogen', 'oil', 'solid_fuel', 'gas', 'electricity', 'biomass']))
    df['service'] = service
    df.service = df.service.astype(CategoricalDtype(['boiler', 'district_heating', 'non_heating', 'heat_pumps', 'fuel_cell']))
    df['sector'] = sector
    df.sector = df.sector.astype(CategoricalDtype(['industry', 'service', 'residential']))
    dfs.append(df)
energy_demand = pandas.concat(dfs)
del dfs
energy_demand.head()

In [ ]:
energy_demand.dtypes

In [ ]:
energy_demand.to_parquet('nic_energy_demand.parquet')

## Transport energy

In [ ]:
def hours_to_int(h):
    """Convert from string-named hours to 24-hour clock integers
    """
    lu = {
        'MIDNIGHT': 0,
        'ONEAM': 1,
        'TWOAM': 2,
        'THREEAM': 3,
        'FOURAM': 4,
        'FIVEAM': 5,
        'SIXAM': 6,
        'SEVENAM': 7,
        'EIGHTAM': 8,
        'NINEAM': 9,
        'TENAM': 10,
        'ELEVENAM': 12,
        'NOON': 11,
        'ONEPM': 13,
        'TWOPM': 14,
        'THREEPM': 15,
        'FOURPM': 16,
        'FIVEPM': 17,
        'SIXPM': 18,
        'SEVENPM': 19,
        'EIGHTPM': 20,
        'NINEPM': 21,
        'TENPM': 22,
        'ELEVENPM': 23,
    }
    return lu[h]

In [ ]:
ev_paths = glob.glob("../results/nic_ed_tr/transport/decision_0/*vehicle*")
dfs = []
for fn in ev_paths:
    output = re.search("output_(\w+)_timestep", fn).group(1)
    year = re.search("2\d+", fn).group(0)
    df = pandas.read_parquet(fn).rename(columns={
        output: 'value'
    })
    df['timestep'] = int(year)
    df['key'] = output
    dfs.append(df)
    
ev_demand = pandas.concat(dfs) \
    .reset_index()
del dfs

ev_demand.annual_day_hours = ev_demand.annual_day_hours.apply(hours_to_int)
ev_demand = ev_demand \
    .pivot_table(
        index=['timestep', 'lad_gb_2016', 'annual_day_hours'], 
        columns='key', 
        values='value'
    ) \
    .reset_index()

del ev_demand.columns.name

ev_demand.head()    

In [ ]:
ev_demand.dtypes

In [ ]:
ev_demand.to_parquet('nic_ev_demand.parquet')

## Transport trips

In [ ]:
tr_data_path = "../results/nic_ed_tr/transport-raw_data_results_nic_ed_tr/"

In [ ]:
# 2015 estimated tempro OD
tempro15 = pandas.read_csv(tr_data_path + "data/csvfiles/temproMatrixListBased198WithMinor4.csv")
tempro15

In [ ]:
# 2015 aggregated LAD OD
lad15 = pandas.read_csv(tr_data_path + "data/csvfiles/ladFromTempro198ODMWithMinor4.csv") \
    .sort_values(by=['origin', 'destination'])
lad15

In [ ]:
# 2050 predicted LAD OD - to disaggregate
lad50 = pandas.read_csv(tr_data_path + "output/2050/predictedODMatrix.csv") \
    .melt(id_vars='origin', var_name='destination', value_name='flow') \
    .sort_values(by=['origin', 'destination'])
lad50

In [ ]:
# tempro zones shapefile - with LAD codes already attached
tempro_lad = geopandas.read_file(tr_data_path + "data/shapefiles/tempro2.shp") \
    .rename(columns={
        'Zone_Name': 'tempro_name',
        'Zone_Code': 'tempro',
        'LAD_Code': 'lad',
        'Local_Auth': 'lad_name'
    }) \
    [['lad', 'lad_name', 'tempro', 'tempro_name']] \
    .sort_values(by=['lad', 'tempro'])
tempro_lad_codes = tempro_lad[['lad', 'tempro']]
tempro_lad

In [ ]:
# start with tempro 2015 OD

# merge on LAD codes for tempro origins
df = tempro15 \
    .rename(columns={'flow': 'tempro2015'}) \
    .merge(tempro_lad_codes, left_on='origin', right_on='tempro') \
    .drop(columns='tempro') \
    .rename(columns={'lad': 'origin_lad'})

# merge on LAD codes for tempro destinations
df = df \
    .merge(tempro_lad_codes, left_on='destination', right_on='tempro') \
    .drop(columns='tempro') \
    .rename(columns={'lad': 'destination_lad'})

# merge on LAD 2015 flows
df = df \
    .merge(lad15, left_on=['origin_lad', 'destination_lad'], right_on=['origin', 'destination'], suffixes=('', '_y')) \
    .drop(columns=['origin_y', 'destination_y']) \
    .rename(columns={'flow': 'lad2015'})

# merge on LAD 2050 flows
df = df \
    .merge(lad50, left_on=['origin_lad', 'destination_lad'], right_on=['origin', 'destination'], suffixes=('', '_y')) \
    .drop(columns=['origin_y', 'destination_y']) \
    .rename(columns={'flow': 'lad2050'})

df

In [ ]:
# Disaggregation calculation
df['tempro2050'] = (df.tempro2015 * (df.lad2050 / df.lad2015)) \
    .round() \
    .astype(int)

In [ ]:
# Quick check
df[(df.origin_lad == 'E09000007') & (df.destination_lad == 'E09000029')]

In [ ]:
df = df.drop(columns=['lad2015', 'lad2050', 'origin_lad', 'destination_lad'])
df

In [ ]:
df.to_parquet('nic_transport_trips.parquet')